# Simple Watch
* Constructs the popularity predictor

In [ ]:
medium = "";
metric = "";

In [ ]:
import NBInclude: @nbinclude
@nbinclude("../Alpha.ipynb");

In [ ]:
const name = "$medium/$metric/Simple"
set_logging_outdir(name);

## Training

In [ ]:
function get_counts(split, metric, medium, col)
    data = getfield(get_split(split, metric, medium, [col]), col)
    counts = Dict{Int32,Int32}()
    @showprogress for i = 1:length(data)
        u = data[i]
        if u ∉ keys(counts)
            counts[u] = 0
        end
        counts[u] += 1
    end
    counts
end;

In [ ]:
const counts = get_counts("training", metric, medium, :itemid)
const total = sum(values(counts))
const popularity = Dict{Int32,Float32}()
for (k, v) in counts
    popularity[k] = v / total
end;

## Inference

In [ ]:
function make_prediction(users, items, p)
    r = zeros(Float32, length(users))
    @showprogress for i = 1:length(r)
        if items[i] in keys(p)
            r[i] = p[items[i]]
        end
    end
    r
end;

In [ ]:
model(users, items) = make_prediction(users, items, popularity);

In [ ]:
write_params(Dict("popularity" => popularity), name);

In [ ]:
write_alpha(model, medium, name, ["test", "negative"]);

In [ ]:
print_losses([name], metric, medium, ["test"]);